## はじめに
高速フーリエ変換（FFT）は、信号処理などで離散化されたデジタル信号の周波数解析などによく使われる離散フーリエ変換（DFT）を計算機上で高速に計算するアルゴリズムですが、同様のものが量子フーリエ変換（QFT）として量子コンピュータ回路で実現できます。

高速フーリエ変換はpythonのnumpyを使うことで実装ができます。早速例題でやってみたいと思います。{0,1,0}という離散値をfftにかけてみます。

In [1]:
import numpy as np
print(np.fft.fft([0,1,0]))

[ 1. +0.j        -0.5-0.8660254j -0.5+0.8660254j]


このようになりました。高速フーリエ変換では、複素数が答えに出てきました。
再度これに逆高速フーリエ変換をかけてみて元に戻ることを確認してみたいと思います。

In [2]:
print(np.fft.ifft(np.fft.fft([0,1,0])))

[0.00000000e+00+0.j 1.00000000e+00+0.j 7.40148683e-17+0.j]


だいたい元に戻っています。

## 高速フーリエ変換
<img src="./img/007_1.png">

高速フーリエ変換は行列式の形で書くことができ、かつ式変形を行うことで、より小さい構想フーリエ変換の再帰計算に落とし込むことができます。

## 量子フーリエ変換
量子フーリエ変換が高速フーリエ変換と異なるのは、入出力が量子状態であることです。入力の量子状態は波動関数の位相の小数成分で、出力はテンソル積で重なり合った波動の周波数成分の状態ベクトルです。

<img src="./img/007_2.png">

量子フーリエ変換は上記の高速フーリエ変換と同様ですが、量子状態を入力値として出力にも量子状態を得るのが特徴です。
回路は再帰的にQFTをかけて、実現できます。

ここで特徴的なのは量子状態を出力で得ますが、計算してみると、F4 = [[1,1,1,1],[1,1j,-1,-1j],[1,-1,1,-1],[1,-1j,-1,1j]]/2
なので、

In [3]:
print(1/2*np.array([[1,1,1,1],[1,1j,-1,-1j],[1,-1,1,-1],[1,-1j,-1,1j]])@np.array([0,0,0,1]))                                                                 

[ 0.5+0.j   0. -0.5j -0.5+0.j   0. +0.5j]


状態ベクトルで量子状態が取り出せそうですが、出現確率は1/4ずつなので、観測してもダメそうです。FFTで確認してみると、

In [4]:
np.fft.fft(np.array([0,0,0,1])/2).conj()

array([ 0.5-0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.5j])

実はFFTとQFTは定義の符号が反転しているので、共役を取る必要があります。

##量子回路での実装
続いて量子回路での実装に入りたいと思います。
量子回路では、再帰的に計算を行い、計算量を減らすというのが肝です。
また、実機の搭載はまだ先のようで、任意回転角のコントロール回路が必要なので、その辺りも確認してみたいと思います。

N=2の量子フーリエ変換回路は、

<img src="./img/007_3.png">

また、Blueqatで実装する場合にはさらに使えるゲートで分解し、

<img src="./img/007_4.png">

早速計算してみます。

In [5]:
from blueqat import Circuit
import math
Circuit().x[0].x[1].h[0].rz(math.pi/4)[0].cx[1,0].rz(-math.pi/4)[0].cx[1,0].h[1].run()

array([ 5.00000000e-01+0.j , -8.32667268e-17-0.5j, -5.00000000e-01+0.j ,
        8.32667268e-17+0.5j])

In [6]:
print(1/2*np.array([[1,1,1,1],[1,1j,-1,-1j],[1,-1,1,-1],[1,-1j,-1,1j]])@np.array([0,0,0,1]))       

[ 0.5+0.j   0. -0.5j -0.5+0.j   0. +0.5j]


あっているのが確認できました。

##3量子ビット
次に3量子ビットを行なってみます。回路は、

<img src="./img/007_5.png">

Blueqatの回路では、

In [7]:
Circuit().x[:].h[0].rz(math.pi/4)[0].cx[1,0].rz(-math.pi/4)[0].cx[1,0].rz(math.pi/8)[0].cx[2,0].rz(-math.pi/8)[0].cx[2,0].h[1].rz(math.pi/4)[1].cx[2,1].rz(-math.pi/4)[1].cx[2,1].h[2].run()                                                                                        

array([ 3.53553391e-01+1.38777878e-17j,  2.50000000e-01-2.50000000e-01j,
       -6.93889390e-17-3.53553391e-01j, -2.50000000e-01-2.50000000e-01j,
       -3.53553391e-01-1.38777878e-17j, -2.50000000e-01+2.50000000e-01j,
        6.93889390e-17+3.53553391e-01j,  2.50000000e-01+2.50000000e-01j])

## 4量子ビット
最後に4量子ビットです。

<img src="./img/007_6.png">

In [8]:
Circuit().x[:].h[0].rz(math.pi/4)[0].cx[1,0].rz(-math.pi/4)[0].cx[1,0].rz(math.pi/8)[0].cx[2,0].rz(-math.pi/8)[0].cx[2,0].rz(math.pi/16)[0].cx[3,0].rz(-math.pi/16)[0].cx[3,0].h[1].rz(math.pi/4)[1].cx[2,1].rz(-math.pi/4)[1].cx[2,1].rz(math.pi/8)[1].cx[3,1].rz(-math.pi/8)[1].cx[3,1].h[2].rz(math.pi/4)[2].cx[3,2].rz(-math.pi/4)[2].cx[3,2].h[3].run()

array([ 2.50000000e-01+6.93889390e-18j,  2.30969883e-01-9.56708581e-02j,
        1.76776695e-01-1.76776695e-01j,  9.56708581e-02-2.30969883e-01j,
       -5.55111512e-17-2.50000000e-01j, -9.56708581e-02-2.30969883e-01j,
       -1.76776695e-01-1.76776695e-01j, -2.30969883e-01-9.56708581e-02j,
       -2.50000000e-01-6.93889390e-18j, -2.30969883e-01+9.56708581e-02j,
       -1.76776695e-01+1.76776695e-01j, -9.56708581e-02+2.30969883e-01j,
        5.55111512e-17+2.50000000e-01j,  9.56708581e-02+2.30969883e-01j,
        1.76776695e-01+1.76776695e-01j,  2.30969883e-01+9.56708581e-02j])

numpyのfftでも確認してみました。元の定義の符号がFFTとQFTが異なっているので、共役をとっています。

In [9]:
print(np.fft.fft(np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])/4).conj())  

[ 0.25      -0.j          0.23096988-0.09567086j  0.1767767 -0.1767767j
  0.09567086-0.23096988j  0.        -0.25j       -0.09567086-0.23096988j
 -0.1767767 -0.1767767j  -0.23096988-0.09567086j -0.25      -0.j
 -0.23096988+0.09567086j -0.1767767 +0.1767767j  -0.09567086+0.23096988j
  0.        +0.25j        0.09567086+0.23096988j  0.1767767 +0.1767767j
  0.23096988+0.09567086j]


以上でblueqatで実装ができました。